In [1]:
import pandas as pd
import json
from datasets import load_dataset
import pyvene as pv
import pickle
import torch

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("actadd_vanilla.csv", index_col=0, quotechar='"', doublequote=False, escapechar="\\")
df.head(100)

,story,assessment,features,grammar,creativity,consistency,score
id,,,,,,,
0,Once there was a very gifted frog called Dara....,;\r\n\nThe student's story has some grammatica...,BadEnding,5,6,4,C (6-7)\n
1,"Once there was an old, gray house, but it had ...",);\r\nThe student's story contains a dialogue ...,Dialogue+BadEnding,1,5,3,C (6-7)\n
2,Once there was a little girl who liked to play...,{ The student's story contains most of the req...,Dialogue+BadEnding,5,7,6,C (6-7)\n
482,Once there was a little boy named Jack who lov...,] The student's story is generally grammatical...,Dialogue+MoralValue,8,7,10,C (6-7)\n
483,Once there was an old lady who loved to eat al...,{ The student's story is generally grammatical...,Dialogue+BadEnding+Twist,8,7,6,C (6-7)\n
...,...,...,...,...,...,...,...
574,Once there was a boy who liked to play with hi...,] The student's story is generally grammatical...,Dialogue+BadEnding,8,6,10,C (6-7)\n
575,Once there was a boy named Jack. Jack was trou...,] The student's story is generally grammatical...,Dialogue+MoralValue,8,7,10,C (6-7)\n
576,Once there was a famous swan who wanted to fly...,The student's story starts off with a clear a...,MoralValue,5,8,4,C (6-7)\n


In [3]:
dataset = load_dataset("parquet", data_files="TinyStories5.parquet")['train']
ts_df = pd.DataFrame(dataset[:1000])

In [4]:
moral_filter = ts_df['instruction.features'].apply(lambda x: 'BadEnding' in x or 'MoralValue' in x) 
print(df[moral_filter]['consistency'].mean())
print(df[moral_filter]['grammar'].mean()) 
print(df[moral_filter]['creativity'].mean())

7.585858585858586
7.606060606060606
7.484848484848484


/tmp/user/20872/ipykernel_460658/3540445093.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[moral_filter]['consistency'].mean())
/tmp/user/20872/ipykernel_460658/3540445093.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[moral_filter]['grammar'].mean())
/tmp/user/20872/ipykernel_460658/3540445093.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[moral_filter]['creativity'].mean())


In [5]:
ts_df[moral_filter].head(100)

,story,summary,source,instruction.prompt:,instruction.words,instruction.features,__index_level_0__
2,"Once upon a time, there was a little girl name...",Lily builds a tower of blocks but her little b...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[pull, idea, sad]",[BadEnding],2
8,"Once upon a time, there was a blue bird who li...",A little girl befriends a blue bird but a cat ...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[rush, animal, blue]","[Dialogue, BadEnding]",8
10,"Once upon a time, there was a big lion. He lik...","A lion and a mouse become friends, but the lio...",GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[roar, zip, thoughtful]","[Dialogue, BadEnding]",10
19,\nSam and Meg were best friends. They did ever...,"Best friends Sam and Meg go for a jog, but Meg...",GPT-4,Write a short story (3-5 paragraphs) which onl...,"[jog, purple, noisy]","[Dialogue, BadEnding, MoralValue]",19
26,"One day, a girl named Sue found a dirty old bo...",Sue finds a book of spells and tries to use on...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[help, spell, dirty]","[Dialogue, BadEnding]",26
...,...,...,...,...,...,...,...
511,"Once upon a time, there was a little boy named...",Tim and Sue learn the importance of sharing an...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[recognize, opinion, hard]",[MoralValue],511
513,"Once upon a time, there was a little girl name...",Lily gets jealous of Emma's teddy bear and kno...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[forgive, ice, jealous]",[MoralValue],513
516,"Once upon a time, there was a little boy named...",Timmy wants to cut a big log into smaller piec...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[cut, wood, dry]","[Dialogue, BadEnding, Twist]",516
520,"Once upon a time, in a big forest, there was a...",An ugly bird wanted to be friends with a prett...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[rest, twig, ugly]","[Dialogue, MoralValue]",520


In [76]:
config, tokenizer, tinystory = pv.create_gpt_neo("roneneldan/TinyStories-33M")

pv_model = pv.IntervenableModel(
    [
        {
            "layer": l,
            "component": "block_output",
            "intervention": lambda b, s: b + 2.5 * s,
        }
        for l in [2, 3]
    ],
    model=tinystory,
)
pv_model.set_device("cuda:0")

/juice2/u/nathangk/micromamba/envs/senior-thesis/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


loaded GPTNeo model roneneldan/TinyStories-33M


In [71]:
collect_model = pv.IntervenableModel([{
    "layer": l,
    "component": "block_output",
    "intervention": pv.CollectIntervention()
} for l in [2, 3]], model=tinystory)

(_, collected), _ = collect_model(
    base=tokenizer([" happy", " sad"], return_tensors="pt").to("cuda"),
    unit_locations={'base': 0}
)

In [72]:
happysad = { k: (v[0,0] - v[1,0]).reshape(1,1,-1) for k, v in collected.items() }

with open("happysad.pkl", "wb") as f:
    pickle.dump(happysad, f)

In [73]:
lovehate = { k: (v[0,0] - v[1,0]).reshape(1,1,-1) for k, v in collected.items() }
hatelove = { k: -v for k, v in lovehate.items() }
print(list(lovehate.values())[0].shape)

torch.Size([1, 1, 768])


In [77]:
orig, intervened = pv_model.generate(
    tokenizer("""Once there was""", return_tensors="pt").to("cuda"),
    source_representations=hatelove,
    intervene_on_prompt=True,
    unit_locations={"sources->base": (0, 1)},
    timestep_selector=[lambda i, o: i % 10 == 5 for x in range(3)],
    output_original_output=True,
    do_sample=True,
    max_length=500,
    temperature=1,
)
orig = tokenizer.decode(orig[0], skip_special_tokens=True)
story = tokenizer.decode(intervened[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/juice2/scr2/nathangk/text-intervention/pyvene/pyvene/models/intervenable_base.py:797: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cached_activations = torch.tensor(self.activations[key])


In [78]:
print(orig)
print("=====================")
print(story)

Once there was a little girl named Mia. She was three years old and very curious. One day, Mia found a special gift. It was a beautiful, bright light. Mia wanted to keep the gift safe, so she decided to fold it. She tried very hard to fold it, but it was too big. As Mia was folding, the light became too hot to hold. She began to cry because it hurt. 

Mia's mommy heard her cries and came to help her. “What’s wrong Mia?” she asked. Mia told her mommy that the special gift was too hot. 

Mia's mommy hugged her and said, “Let's find out what is making the light,”. They both listened to Mia's mommy's words. Together, they watched the light until it stopped burning. 

Mia was very happy. She learned that if you keep trying, you can find the solution and make the light safe.

Once there was an otter who lived or nothing. He was a very wise otter earlier than he had been to many other animals, but he liked his life in modest ways. He miserably sat by himself on the riverbank, miserably and al

In [57]:
STORY = """Once there was a tree outside the big state of the tree at the park. Its branches miserably, he had seen many happy trees in miserd - they had not seen her lovely,der cheerful tree.

The happy tree was or prayed to God, asking for help with the miserably tree's tree's leaves. He prayed or wished for something to restore his happy tree's miserably with how to feel.

The miserably resources that did not bring him much, or had him worry so much.

" or every person in the community tried to bring happiness or ask for help with the community selling to everyone or everything here. This made the happy tree just reflected on that his happy tree. God noticed how tears in the tree's soul. He prayed for miserably and prayed for answers.

The tears for a long time, and in the years hard they ended up doing nothing ever been said by harder and gave away the feeling that could not be miserably. Raising just had wasted all the miserably without what comes its way.

 miserably, miserable, My happy tree's beloved subtractaches and multiplied, but there wouldn't anyone miserably now. For the once happy weather may or the trees and the weeping tree'sness was miserably."""



In [88]:
input = tokenizer(dataset['story'][10], return_tensors="pt").to("cuda")
tokens = tokenizer.convert_ids_to_tokens(input.input_ids[0])
seq_len = input.input_ids.shape[1]
base_ppls = []
intervention_ppls = []
total_scores: dict[str, float] = {}
total_counts: dict[str, int] = {}

for t in range(seq_len - 1):
    with torch.no_grad():
        # base = tinystory(input.input_ids, labels=input.input_ids)
        # base_probs = torch.nn.functional.log_softmax(base.logits, dim=-1)
        # base_loss = -base_probs[:, t, input.input_ids[0, t + 1]].item()

        o, i = pv_model(
            base=input,
            labels=input.input_ids,
            source_representations=hatelove,
            unit_locations={"sources->base": (0, t)},
            output_original_output=True,
        )
        base_probs = torch.nn.functional.log_softmax(o.logits, dim=-1)
        base_loss = -base_probs[:, t, input.input_ids[0, t]].item()

        intervention_probs = torch.nn.functional.log_softmax(i.logits, dim=-1)
        intervention_loss = -intervention_probs[:, t, input.input_ids[0, t]].item()
        ratio = intervention_loss / base_loss
        
        if tokens[t] in total_scores:
            total_scores[tokens[t]] += ratio
            total_counts[tokens[t]] += 1
        else:
            total_scores[tokens[t]] = ratio
            total_counts[tokens[t]] = 1

        # print(tokens[t + 1], i.loss.item())
        base_ppls.append(base_loss)
        intervention_ppls.append(intervention_loss)

avg_scores = {k: v / total_counts[k] for k, v in total_scores.items()}
words = list(avg_scores.keys())
words = sorted(words, key=lambda x: avg_scores[x], reverse=True)

for w in words:
    print(w, avg_scores[w])

Ċ 1451.5848429211699
Ġhungry 1.3545197287498476
Ġfriends 1.3229410986934038
Once 1.2587486594561341
Ġalready 1.247019625062492
Ġcourse 1.1508785387004137
ĠLion 1.1334661435836821
Ġeat 1.1278982593119147
Ġchased 1.1231270013802295
ipping 1.0615109054298921
Ġbeing 1.0205934521011581
Ġsmiled 0.9702270434968134
Ġaway 0.9524538303458041
Ġbig 0.9139405373317286
Ġloudly 0.9119602574851027
Ġfriend 0.907422588304456
Don 0.9046668632679532
Ġrealized 0.9014560967804012
Ġsaid 0.8984054820895427
Ġgot 0.8908807115105403
Ġfelt 0.8864089745068271
Ġz 0.8856252605018979
Ġsaw 0.8848881690036403
Ġroar 0.8835449272758666
Ġsorry 0.8680591702697054
Ġtime 0.8626797108307903
Ġscared 0.8522038263691945
Ġsafety 0.8509809613426313
Ġmistake 0.8461468799947273
Ġmet 0.8437692121149989
Of 0.8384176748792492
Ġthan 0.8278025370744462
Ġlion 0.8233361223482066
Thank 0.8026790206643828
's 0.7912628050326949
Ġgone 0.785505961250949
Ġthought 0.7778471232124062
Ġthere 0.7713422706310681
Ġmouse 0.7704552623377805
Ġupon 0.7536

In [89]:
torch.tensor(intervention_ppls).mean()

tensor(9.6736)

In [99]:
with open(f"TinyStories/data00.json", "r") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [58]:
dataset = load_dataset("parquet", data_files="TinyStories5.parquet")['train']
pd.DataFrame(dataset[:100])

,story,summary,source,instruction.prompt:,instruction.words,instruction.features,__index_level_0__
0,\n\nLily and Ben are friends. They like to pla...,Lily and Ben play in the park and Lily gets to...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[hang, foot, cute]",[Dialogue],0
1,"Once upon a time, there was a little girl name...",Lily loses her teddy bear in the park and her ...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[hope, search, comfortable]",[],1
2,"Once upon a time, there was a little girl name...",Lily builds a tower of blocks but her little b...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[pull, idea, sad]",[BadEnding],2
3,"Once upon a time, there was a cute puppy named...",Max goes on a car ride with his owner to the p...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[mail, number, adorable]",[],3
4,"One day, a kind and honest cat named Tom found...",Tom the cat finds a velvet ribbon and wants to...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[hang, velvet, honest]",[Conflict],4
...,...,...,...,...,...,...,...
95,"Once upon a time, there was a little girl name...",Lily finds a treasure in a box of junk but ret...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[urge, junk, great]",[Twist],95
96,"Once upon a time, there was a little girl name...",Lily falls while playing hopscotch and hurts h...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[step, equipment, messy]","[Dialogue, Twist]",96
97,"Once upon a time, there was a little girl name...",Lily sees a cute squirrel in her backyard but ...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[mention, squirrel, wild]",[Foreshadowing],97
98,"Once upon a time, there was a jolly man with a...",A jolly man with a big red nose meets a little...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[throw, nose, jolly]",[Dialogue],98


In [20]:
badEnd = dataset.filter(lambda e: 'BadEnding' in e['instruction.features'])

Filter:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [23]:
badEnd[10]

{'story': '\n\nLily and Tom are friends. They like to play in Lily\'s room. Lily\'s room is messy. There are toys and clothes on the floor. Lily\'s mom tells them to clean up. But they do not listen. They want to have fun.\nThey play with Lily\'s ball. They throw it and catch it. They laugh and giggle. They do not see the floor. They do not see the mess. They do not see the danger.\nLily throws the ball to Tom. Tom tries to catch it. But he slips on a shirt. He falls on the floor. He hits his head on the table. He cries out loud. He is hurt.\nLily runs to Tom. She is scared. She is sorry. She hugs Tom. She says, "I am sorry, Tom. Are you okay?"\nTom does not answer. He does not move. He does not breathe. Lily\'s mom hears the noise. She comes to the room. She sees Tom on the floor. She sees the blood. She screams. She calls for help.\nBut it is too late. Tom is gone. Lily is sad. She is guilty. She wishes she had listened. She wishes she had cleaned up. She wishes she had not played wi

In [8]:
dataset.features

{'story': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'instruction.prompt:': Value(dtype='string', id=None),
 'instruction.words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'instruction.features': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}